In [5]:
import pandas as pd
from node2vec import Node2Vec
import networkx as nx

In [2]:
model_dataset = pd.read_parquet("model_dataset.parquet")

In [6]:
gpickle_path = "../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle"
G = nx.read_gpickle(gpickle_path)

In [ ]:
# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(G, dimensions=8, walk_length=5, num_walks=20, workers=16)  # Use temp_folder for big graphs

# Embed nodes
model = node2vec.fit(window=4, min_count=1, )
#batch_words=4) 
# Any keywords acceptable by gensim.Word2Vec can be passed, 
# `dimensions` and `workers` are automatically passed (from the Node2Vec constructor)

Computing transition probabilities:   0%|          | 0/55492 [00:00<?, ?it/s]

In [ ]:
index_to_key = model.wv.index_to_key
normed_vectors = model.wv.get_normed_vectors()
normed_vectors.shape

In [ ]:
n2v_features = [f"n2v_{i}" for i in range(normed_vectors.shape[1])]
n2v_embd = pd.DataFrame(normed_vectors, columns=n2v_features)
n2v_embd['id'] = pd.Series(model.wv.index_to_key).astype(int)
model_dataset = pd.merge(
    model_dataset, 
    n2v_embd,
    on='id'
)

In [12]:
model_dataset.shape

(431921, 47)

In [ ]:
model_dataset.reset_index(drop=True).to_parquet("model_dataset.parquet")